# 🚦 YOLO Detection Server V15 - PARALLEL PIPELINE FIXED
### Fixed: Stream timeout, model loading like V12
### Async Pipeline: Reader → Detector → Sender

In [ ]:
# ═══════════════════════════════════════════════════════════
# Cell 1: CONFIGURATION
# ═══════════════════════════════════════════════════════════

BACKEND_HTTP_URL = 'https://teens-relief-performed-blank.trycloudflare.com'
BACKEND_WS_URL = 'wss://kevin-revolution-fish-foster.trycloudflare.com'
NMS_URL = 'https://designer-supply-later-landing.trycloudflare.com'

TRACK_LINE_Y = 50
CONFIDENCE = 0.5
IMGSZ = 640
USE_HALF = True

# Queue sizes
FRAME_QUEUE_SIZE = 3
RESULT_QUEUE_SIZE = 10

print(f"📡 HTTP: {BACKEND_HTTP_URL}")
print(f"📡 WS: {BACKEND_WS_URL}")
print(f"📺 NMS: {NMS_URL}")

In [ ]:
# Cell 2: Install Dependencies (Fix PIL & Numpy)
# Uninstall conflicting packages first
!pip uninstall -y numpy pillow ultralytics > /dev/null 2>&1

# Install stable versions
!pip install "numpy<2.0.0" "pillow>=10.3.0" scipy ultralytics opencv-python-headless requests websocket-client --upgrade --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt

import numpy, PIL
print(f'✅ Dependencies installed. Numpy: {numpy.__version__}, PIL: {PIL.__version__}')

In [ ]:
# Cell 3: Load Models (Using Kaggle Dataset Paths) - SAME AS V12
import warnings
warnings.filterwarnings('ignore')
import os, torch
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'🚀 Device: {device}')

vehicle_model = traffic_light_model = lp_detector = char_detector = None
base_dir = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/2/'

# Vehicle Detection
try:
    vehicle_model = YOLO('yolo11m.pt').to(device)
    print('✅ Vehicle')
except Exception as e: print(f'⚠️ Vehicle: {e}')

# Traffic Light Detection
try:
    p = os.path.join(base_dir, 'mhiot-dentinhieu-best-new.pt')
    if os.path.exists(p):
        traffic_light_model = YOLO(p).to(device)
        print('✅ Traffic Light')
    else: print(f'⚠️ TL Model not found at {p}')
except Exception as e: print(f'⚠️ TL: {e}')

# Vietnamese License Plate Detection
try:
    p = os.path.join(base_dir, 'license_plate_detector.pt')
    if os.path.exists(p):
        lp_detector = YOLO(p).to(device)
        print('✅ LP Detector (YOLOv8)')
    else: print(f'⚠️ LP Detector not found at {p}')
except Exception as e: print(f'⚠️ LP Detector: {e}')

# Vietnamese Character Detection
try:
    p = os.path.join(base_dir, 'character_detector.pt')
    if os.path.exists(p):
        char_detector = YOLO(p).to(device)
        print('✅ Character Detector (YOLOv8)')
    else: print(f'⚠️ Char Detector not found at {p}')
except Exception as e: print(f'⚠️ Char Detector: {e}')

print(f"\n📊 V:{'✅' if vehicle_model else '❌'} TL:{'✅' if traffic_light_model else '❌'} LP:{'✅' if lp_detector else '❌'} OCR:{'✅' if char_detector else '❌'}")

In [ ]:
# Cell 4: Detection Functions - SAME AS V12
import cv2, numpy as np, time, requests, threading, re, queue, json
from datetime import datetime
from collections import deque

VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']
camera_trackers = {}

def log(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

def read_plate_characters(plate_img):
    if char_detector is None or plate_img.size == 0:
        return 'unknown'
    try:
        results = char_detector(plate_img, verbose=False)
        chars = []
        for r in results:
            for b in r.boxes:
                if float(b.conf[0]) < 0.3: continue
                x1 = int(b.xyxy[0][0])
                cls_id = int(b.cls[0])
                char_val = char_detector.names[cls_id]
                chars.append((x1, char_val))
        if len(chars) < 5: return 'unknown'
        chars.sort(key=lambda x: x[0])
        text = ''.join([c[1] for c in chars])
        text = text.upper().replace(' ', '').replace('-', '')
        return text
    except: return 'unknown'

def detect_license_plate(frame, vehicle_bbox):
    if lp_detector is None: return None
    x1, y1, x2, y2 = vehicle_bbox
    vehicle_crop = frame[y1:y2, x1:x2]
    if vehicle_crop.size == 0: return None
    try:
        results = lp_detector(vehicle_crop, verbose=False)
        best_plate = None
        max_conf = 0
        for r in results:
            for b in r.boxes:
                conf = float(b.conf[0])
                if conf > max_conf and conf > 0.3:
                    max_conf = conf
                    best_plate = b
        if best_plate:
            px1, py1, px2, py2 = map(int, best_plate.xyxy[0])
            plate_crop = vehicle_crop[py1:py2, px1:px2]
            text = read_plate_characters(plate_crop)
            if text != 'unknown':
                return {
                    'text': text,
                    'confidence': max_conf,
                    'bbox_pixels': [x1+px1, y1+py1, x1+px2, y1+py2]
                }
    except: pass
    return None

def detect_frame(frame, camera_id):
    h, w = frame.shape[:2]
    result = {
        'camera_id': camera_id,
        'created_at': int(time.time() * 1000),
        'image_dimensions': {'width': w, 'height': h},
        'track_line_y': TRACK_LINE_Y
    }
    if camera_id not in camera_trackers:
        camera_trackers[camera_id] = {
            'tracks': {}, 'counted': {},
            'counts_up': {v:0 for v in VEHICLE_CLASSES},
            'counts_down': {v:0 for v in VEHICLE_CLASSES},
            'total_up': 0, 'total_down': 0
        }
    tr = camera_trackers[camera_id]
    debug_parts = []
    
    # Vehicle Detection
    if vehicle_model:
        t0 = time.time()
        dets, tracks, vcounts, crossings = [], {}, {v:0 for v in VEHICLE_CLASSES}, []
        ly = int(h * TRACK_LINE_Y / 100)
        
        for r in vehicle_model.track(frame, persist=True, verbose=False, imgsz=IMGSZ, conf=CONFIDENCE):
            for b in r.boxes:
                cls = vehicle_model.names[int(b.cls[0])]
                if cls not in VEHICLE_CLASSES or float(b.conf[0]) < CONFIDENCE: continue
                x1,y1,x2,y2 = map(int, b.xyxy[0])
                cx, cy = (x1+x2)//2, (y1+y2)//2
                det = {
                    'class': cls, 'type': 'vehicle', 'confidence': float(b.conf[0]),
                    'bbox': {'x1': x1/w, 'y1': y1/h, 'x2': x2/w, 'y2': y2/h, 'width': (x2-x1)/w, 'height': (y2-y1)/h},
                    'bbox_pixels': [x1, y1, x2, y2]
                }
                if hasattr(b,'id') and b.id is not None:
                    tid = int(b.id[0]); det['id'] = tid
                    tracks[tid] = {'pos': (cx,cy), 'time': result['created_at'], 'class': cls}
                
                # License Plate Detection
                lp = detect_license_plate(frame, [x1, y1, x2, y2])
                if lp: 
                    det['license_plate'] = lp
                    debug_parts.append(f"LP:{lp['text']}")
                
                dets.append(det); vcounts[cls] += 1
        
        for tid, info in tracks.items():
            if tid not in tr['tracks']: tr['tracks'][tid] = []
            if tr['tracks'][tid]:
                py, cy = tr['tracks'][tid][-1]['pos'][1], info['pos'][1]
                d = 1 if py <= ly < cy else (-1 if py >= ly > cy else 0)
                if d and f"{tid}_{d}" not in tr['counted']:
                    tr['counted'][f"{tid}_{d}"] = True
                    if d == 1: tr['counts_down'][info['class']] += 1; tr['total_down'] += 1
                    else: tr['counts_up'][info['class']] += 1; tr['total_up'] += 1
                    crossings.append({'id': tid, 'direction': d})
            tr['tracks'][tid].append({'pos': info['pos'], 'time': info['time'], 'class': info['class']})
            tr['tracks'][tid] = tr['tracks'][tid][-30:]
        
        result['vehicle'] = {
            'detections': dets, 'inference_time': (time.time()-t0)*1000,
            'vehicle_count': {
                'total_up': tr['total_up'], 'total_down': tr['total_down'],
                'by_type_up': tr['counts_up'].copy(), 'by_type_down': tr['counts_down'].copy(),
                'current': vcounts
            },
            'new_crossings': crossings,
            'tracks': [{'id': tid, 'positions': [{'x': p['pos'][0], 'y': p['pos'][1], 'time': p['time']} for p in tdata], 'class': tdata[-1]['class']} for tid, tdata in tr['tracks'].items() if tdata and tid in tracks]
        }
        veh_summary = ','.join([f"{k}:{v}" for k,v in vcounts.items() if v > 0])
        if veh_summary: debug_parts.append(f"Veh[{veh_summary}]")
    
    # Traffic Light Detection
    if traffic_light_model:
        t0 = time.time(); tl_dets = []; status = None; mx = 0
        for r in traffic_light_model(frame, verbose=False):
            for b in r.boxes:
                cf = float(b.conf[0])
                if cf < 0.4: continue
                tx1,ty1,tx2,ty2 = map(int, b.xyxy[0])
                cn = traffic_light_model.names[int(b.cls[0])]
                tl_dets.append({'class': cn, 'confidence': cf, 'bbox': {'x1':tx1/w, 'y1':ty1/h, 'x2':tx2/w, 'y2':ty2/h, 'width':(tx2-tx1)/w, 'height':(ty2-ty1)/h}, 'bbox_pixels': [tx1, ty1, tx2, ty2]})
                if cf > mx: mx, status = cf, cn
        result['traffic_light'] = {'detections': tl_dets, 'traffic_status': status, 'inference_time': (time.time()-t0)*1000}
        if status: debug_parts.append(f"TL:{status}")
    
    if debug_parts:
        log(f"[{camera_id[-4:]}] " + " | ".join(debug_parts))
        
    return result

print('✅ Detection functions ready')

In [ ]:
# Cell 5: WebSocket Client (Non-blocking)
import websocket

class AsyncWebSocket:
    def __init__(self, url, camera_id, api_key):
        self.url = f"{url}?cameraId={camera_id}&apiKey={api_key}"
        self.camera_id = camera_id
        self.ws = None
        self.connected = False
        self.send_queue = queue.Queue(maxsize=RESULT_QUEUE_SIZE)
        self._stop = False

    def connect(self):
        try:
            log(f'🔌 [{self.camera_id[-4:]}] Connecting WS...')
            self.ws = websocket.WebSocketApp(
                self.url,
                on_open=self._on_open,
                on_close=lambda ws,c,m: setattr(self, 'connected', False),
                on_error=lambda ws,e: log(f'⚠️ [{self.camera_id[-4:]}] WS Error: {e}')
            )
            threading.Thread(target=self.ws.run_forever, daemon=True).start()
            threading.Thread(target=self._sender_loop, daemon=True).start()
            # Wait for connection with timeout
            for _ in range(30):  # 3 second timeout
                if self.connected:
                    return True
                time.sleep(0.1)
            return self.connected
        except Exception as e:
            log(f'❌ [{self.camera_id[-4:]}] WS Connect failed: {e}')
            return False
    
    def _on_open(self, ws):
        self.connected = True
        log(f'✅ [{self.camera_id[-4:]}] WS Connected')
    
    def _sender_loop(self):
        while not self._stop:
            try:
                result = self.send_queue.get(timeout=1)
                if self.connected and self.ws:
                    self.ws.send(json.dumps(result))
            except queue.Empty:
                continue
            except Exception as e:
                self.connected = False
    
    def send_async(self, data):
        try:
            self.send_queue.put_nowait(data)
            return True
        except queue.Full:
            try:
                self.send_queue.get_nowait()
                self.send_queue.put_nowait(data)
            except: pass
            return False

print('✅ Async WebSocket ready')

In [ ]:
# Cell 6: Parallel Pipeline - FIXED
import cv2, time, requests, threading, queue
from collections import deque

camera_stats = {}
camera_keys = {}

def fetch_cameras():
    try:
        resp = requests.get(f"{BACKEND_HTTP_URL}/api/camera/all", timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            cams = data.get('metadata', data) if isinstance(data, dict) else data
            for cam in cams:
                if isinstance(cam, dict) and cam.get('_id'):
                    camera_keys[cam['_id']] = cam.get('camera_api_key', '')
            return list(camera_keys.keys())
    except Exception as e: log(f'❌ {e}')
    return []

def test_stream(flv_url, timeout=10):
    """Test if FLV stream is available"""
    cap = cv2.VideoCapture(flv_url)
    cap.set(cv2.CAP_PROP_OPEN_TIMEOUT_MSEC, timeout * 1000)
    cap.set(cv2.CAP_PROP_READ_TIMEOUT_MSEC, timeout * 1000)
    if cap.isOpened():
        ret, _ = cap.read()
        cap.release()
        return ret
    return False

class CameraPipeline:
    def __init__(self, camera_id, api_key):
        self.camera_id = camera_id
        self.flv_url = f"{NMS_URL}/live/{camera_id}.flv"
        self.frame_queue = queue.Queue(maxsize=FRAME_QUEUE_SIZE)
        self.ws = AsyncWebSocket(BACKEND_WS_URL, camera_id, api_key)
        self._stop = False
        self.stats = {'frames_read': 0, 'frames_detected': 0, 'fps': 0}
        self.fps_times = deque(maxlen=30)
        self.stream_ready = False
    
    def start(self):
        # Connect WebSocket first
        if not self.ws.connect():
            log(f'❌ [{self.camera_id[-4:]}] WS Failed, skipping camera')
            return False
        
        # Test stream availability
        log(f'🎥 [{self.camera_id[-4:]}] Testing stream...')
        if not test_stream(self.flv_url, timeout=10):
            log(f'⚠️ [{self.camera_id[-4:]}] Stream not available, will retry in background')
        
        # Start threads
        threading.Thread(target=self._reader_loop, daemon=True, name=f'reader_{self.camera_id[-4:]}').start()
        threading.Thread(target=self._detector_loop, daemon=True, name=f'detector_{self.camera_id[-4:]}').start()
        log(f'🚀 [{self.camera_id[-4:]}] Pipeline started')
        return True
    
    def _reader_loop(self):
        retry_delay = 2
        while not self._stop:
            try:
                log(f'🎥 [{self.camera_id[-4:]}] Connecting to stream...')
                cap = cv2.VideoCapture(self.flv_url)
                cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # Reduce buffer for lower latency
                
                if not cap.isOpened():
                    log(f'❌ [{self.camera_id[-4:]}] Stream not available, retry in {retry_delay}s')
                    time.sleep(retry_delay)
                    retry_delay = min(retry_delay * 2, 30)  # Exponential backoff max 30s
                    continue
                
                log(f'✅ [{self.camera_id[-4:]}] Stream connected!')
                self.stream_ready = True
                retry_delay = 2  # Reset delay on success
                
                while cap.isOpened() and not self._stop:
                    ret, frame = cap.read()
                    if not ret:
                        log(f'⚠️ [{self.camera_id[-4:]}] Frame read failed')
                        break
                    
                    self.stats['frames_read'] += 1
                    
                    # Put frame in queue (non-blocking, drop old if full)
                    try:
                        self.frame_queue.put_nowait(frame)
                    except queue.Full:
                        try:
                            self.frame_queue.get_nowait()  # Drop oldest
                            self.frame_queue.put_nowait(frame)
                        except: pass
                
                cap.release()
                self.stream_ready = False
                
            except Exception as e:
                log(f'❌ [{self.camera_id[-4:]}] Reader error: {e}')
                time.sleep(retry_delay)
    
    def _detector_loop(self):
        while not self._stop:
            try:
                frame = self.frame_queue.get(timeout=2)
                
                result = detect_frame(frame, self.camera_id)
                self.stats['frames_detected'] += 1
                
                self.ws.send_async(result)
                
                # FPS calculation
                now = time.time()
                self.fps_times.append(now)
                if len(self.fps_times) > 1:
                    self.stats['fps'] = round(len(self.fps_times) / (self.fps_times[-1] - self.fps_times[0]), 1)
                
                if self.stats['frames_detected'] % 50 == 0:
                    inf = result.get('vehicle', {}).get('inference_time', 0)
                    det = len(result.get('vehicle', {}).get('detections', []))
                    log(f"[{self.camera_id[-4:]}] {self.stats['fps']} FPS | {inf:.0f}ms | {det} det | Q:{self.frame_queue.qsize()}")
                
            except queue.Empty:
                continue
            except Exception as e:
                log(f'❌ [{self.camera_id[-4:]}] Detector error: {e}')

# Main
log('🔍 Fetching cameras...')
cameras = fetch_cameras()
if not cameras:
    log('❌ No cameras!')
else:
    log(f'✅ {len(cameras)} cameras')
    pipelines = []
    for cid in cameras:
        p = CameraPipeline(cid, camera_keys.get(cid, ''))
        if p.start():
            pipelines.append(p)
        time.sleep(0.5)  # Stagger starts
    
    log(f'🚀 {len(pipelines)} PIPELINES RUNNING!')
    
    try:
        while True:
            time.sleep(60)
            for p in pipelines:
                log(f"📊 [{p.camera_id[-4:]}] Read:{p.stats['frames_read']} Det:{p.stats['frames_detected']} FPS:{p.stats['fps']} Stream:{'✅' if p.stream_ready else '❌'}")
    except KeyboardInterrupt:
        log('Stopping...')